In [3]:
import numpy as np
import tensorflow as tf

In [4]:
class StackGAN():
    def __init__(self,image_dims):
        self.s = image_dims
        self.s2,self.s4,self.s8,self.s16 = self.s//2,self.s//4,self.s//8,self.s//16
        self.gfdim = 128
        
    def get_conv(self,tensor,shape,initializer, name, padding = 'SAME', strides = [1,1,1,1],isrelu = True, isbn = True):
        W = tf.get_variable(name=name,shape=shape,initializer= initializer)
        conv = tf.nn.conv2d(tensor,filter=W,strides=strides,padding = padding)
        if(isbn):
            conv = tf.layers.batch_normalization(conv)
        if(isrelu):
            conv = tf.nn.relu(conv)
        return conv
        
    
    def generator(self,z):
        z = tf.contrib.layers.flatten(inputs=z)
        output_shape = (self.s16**2)*self.gfdim
        W1 = tf.get_variable(initializer=tf.random_normal_initializer(stddev=0.2),name='W1',shape= [z.shape[1],output_shape])
        b1 = tf.get_variable(initializer=tf.constant_initializer(0.0),name = 'b1', shape = [output_shape])
        fc1 = tf.matmul(z,W1) + b1
        bn1 = tf.layers.batch_normalization(fc1,axis=1)
        rs1 = tf.reshape(bn1,shape=[-1,self.s16,self.s16,self.gfdim*8])
        
        Wc1 = tf.get_variable(name='Wc1', initializer=tf.truncated_normal_initializer(stddev=0.02),shape = [1,1,self.gfdim,self.gfdim*2])
        conv1 = tf.nn.conv2d(input=rs1,filter=Wc1,strides = [1,1,1,1],padding = 'SAME')
        bn2 = tf.layers.batch_normalization(conv1)
        rel1 = tf.nn.relu(bn2)
        Wc2 = tf.get_variable(name='Wc2', initializer=tf.truncated_normal_initializer(stddev=0.02), shape=[3,3,self.gfdim*2,self.gfdim*2])
        conv2 = tf.nn.conv2d(input=rel1, filter=Wc2, strides=[1,1,1,1],padding='SAME')
        bn3 = tf.layers.batch_normalisation(conv2)
        rel2 = tf.nn.relu(bn3)
        Wc3 = tf.get_variable(name='Wc3', initializer=tf.truncated_normal_initializer(stddev=0.02), shape=[3,3,self.gfdim*2,self.gfdim*8])
        conv3 = tf.nn.conv2d(input=rel2, filter=Wc3, strides=[1,1,1,1],padding='SAME')
        bn4 = yf.layers.batch_normalisation(conv3)
        
        sum1 = rs1 + gn4
        rel3 = tf.nn.relu(sum1)
        initializer = tf.truncated_normal_initializer(stddev=0.02)
        convT1 = tf.image.resize_nearest_neighbor(size=[self.s8,self.s8])
        Wc4 = tf.get_variable(name='Wc4',initializer=initializer,shape=[3,3,self.gfdim*8,self.gfdim*4])
        conv4 = tf.nn.conv2d(input = convT1, filter = Wc4, strides = [1,1,1,1],padding = 'SAME')
        bn5 = tf.layers.batch_normalization(conv4)
        
        conv5 = self.get_conv(bn5,[1,1,self.gfdim*4,self.gfdim],initializer,'Wc5')
        conv6 = self.get_conv(conv5,[3,3,self.gfdim,self.gfdim],initializer,'Wc6')
        conv7 = self.get_conv(conv6,[3,3,self.gfdim,self.gfdim*4],initializer,'Wc7',isrelu=False)
        sum2 = tf.nn.relu(bn5 + conv7)
        convT2 = tf.image.resize_nearest_neighbor(images=sum2,size=[self.s4,self.s4])
        conv8 = self.get_conv(convT2,[3,3,self.gfdim*4,self.gfdim*2],initializer,'Wc8')
        convT3 = tf.image.resize_nearest_neighbor(images=conv8,size=[self.s2,self.s2])
        conv9 = self.get_conv(convT3,[3,3,self.gfdim*2,self.gfdim],initializer,'Wc9')
        convT4 = tf.image.resize_nearest_neighbor(images=conv9,size=[self.s,self.s])
        conv10 = self.get_conv(convT4,[3,3,self.gfdim,3],initializer,'Wc10')
        image = tf.nn.tanh(conv10)
        return image